# 📊 Análise de Dados Financeiros Yampa

Este notebook contém a análise completa dos dados financeiros reais exportados do sistema Yampa.

## Objetivo:
- Explorar e analisar os arquivos da pasta `data/base`
- Consolidar dados históricos (2020-2025)
- Criar análises financeiras detalhadas
- Gerar insights para tomada de decisão

## Arquivos Disponíveis:
- `yampa_relatorio.csv` - Dados atuais (2022-2025)
- `yampa_relatorio_antigo.csv` - Dados históricos (2020-2022)
- `plano_de_contas_correto.xlsx` - Estrutura do plano de contas

**Total de Transações: 47.064** 💰

## 1. Importar Bibliotecas Necessárias

In [1]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from pathlib import Path
from datetime import datetime, timedelta
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")
print(f"📈 Data e hora atual: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# Definir função para tratar valores brasileiros
def tratar_valor_brasileiro(valor):
    """Converte valores no formato brasileiro para float"""
    if pd.isna(valor) or valor == '' or valor == ' ':
        return 0.0
    
    valor = str(valor).strip()
    if valor in ['0,00', '0', '', ' ']:
        return 0.0
    
    # Tratar formato brasileiro: -3.330,11 -> -3330.11
    if ',' in valor:
        if valor.count(',') == 1:
            partes = valor.split(',')
            parte_inteira = partes[0].replace('.', '')
            parte_decimal = partes[1]
            valor_tratado = parte_inteira + '.' + parte_decimal
        else:
            valor_tratado = valor.replace(',', '.')
    else:
        valor_tratado = valor.replace('.', '')
    
    try:
        return float(valor_tratado)
    except:
        return 0.0

print("🔧 Função de tratamento de valores criada!")

✅ Bibliotecas importadas com sucesso!
📊 Pandas versão: 2.3.3
🔢 NumPy versão: 2.3.3
📈 Data e hora atual: 14/10/2025 08:51:42
🔧 Função de tratamento de valores criada!


## 2. Explorar Estrutura do Diretório

In [2]:
# Explorar estrutura do diretório
print("📁 ESTRUTURA DO DIRETÓRIO")
print("=" * 50)

# Definir caminhos
base_dir = Path("../data")
data_base_dir = base_dir / "base"

print(f"📂 Diretório base: {base_dir.absolute()}")
print(f"📂 Diretório data/base: {data_base_dir.absolute()}")

# Verificar se os diretórios existem
if base_dir.exists():
    print("✅ Diretório data/ encontrado")
    
    if data_base_dir.exists():
        print("✅ Diretório data/base/ encontrado")
        
        # Listar conteúdo do diretório data/
        print(f"\n📋 Conteúdo do diretório data/:")
        for item in base_dir.iterdir():
            if item.is_file():
                size_mb = item.stat().st_size / (1024 * 1024)
                print(f"   📄 {item.name} ({size_mb:.2f} MB)")
            else:
                print(f"   📁 {item.name}/")
    else:
        print("❌ Diretório data/base/ não encontrado")
else:
    print("❌ Diretório data/ não encontrado")

# Verificar sistema operacional
print(f"\n💻 Sistema: {os.name}")
print(f"📍 Diretório atual: {Path.cwd()}")

📁 ESTRUTURA DO DIRETÓRIO
📂 Diretório base: d:\projetos\pulso_finance\notebooks\..\data
📂 Diretório data/base: d:\projetos\pulso_finance\notebooks\..\data\base
✅ Diretório data/ encontrado
✅ Diretório data/base/ encontrado

📋 Conteúdo do diretório data/:
   📁 base/
   📄 exemplo_financeiro.csv (0.00 MB)

💻 Sistema: nt
📍 Diretório atual: d:\projetos\pulso_finance\notebooks


## 3. Listar Arquivos na Pasta data/base

In [3]:
# Listar e analisar arquivos na pasta data/base
print("📋 ARQUIVOS DISPONÍVEIS EM data/base")
print("=" * 60)

if data_base_dir.exists():
    arquivos_info = []
    
    for arquivo in data_base_dir.iterdir():
        if arquivo.is_file():
            # Obter informações do arquivo
            tamanho_bytes = arquivo.stat().st_size
            tamanho_mb = tamanho_bytes / (1024 * 1024)
            extensao = arquivo.suffix.lower()
            modificado = datetime.fromtimestamp(arquivo.stat().st_mtime)
            
            arquivos_info.append({
                'Nome': arquivo.name,
                'Extensão': extensao,
                'Tamanho (MB)': tamanho_mb,
                'Modificado': modificado.strftime('%d/%m/%Y %H:%M'),
                'Caminho': str(arquivo)
            })
            
            # Identificar tipo de arquivo
            if extensao == '.csv':
                tipo = "📊 Arquivo CSV"
            elif extensao in ['.xlsx', '.xls']:
                tipo = "📗 Arquivo Excel"
            elif extensao == '.json':
                tipo = "📄 Arquivo JSON"
            else:
                tipo = "📁 Outro arquivo"
            
            print(f"{tipo}")
            print(f"   Nome: {arquivo.name}")
            print(f"   Tamanho: {tamanho_mb:.2f} MB")
            print(f"   Modificado: {modificado.strftime('%d/%m/%Y %H:%M')}")
            print()
    
    # Criar DataFrame com informações dos arquivos
    if arquivos_info:
        df_arquivos = pd.DataFrame(arquivos_info)
        print("📈 RESUMO DOS ARQUIVOS:")
        print(df_arquivos.to_string(index=False))
        
        print(f"\n📊 ESTATÍSTICAS:")
        print(f"Total de arquivos: {len(arquivos_info)}")
        print(f"Tamanho total: {df_arquivos['Tamanho (MB)'].sum():.2f} MB")
        print(f"Arquivos CSV: {len([a for a in arquivos_info if a['Extensão'] == '.csv'])}")
        print(f"Arquivos Excel: {len([a for a in arquivos_info if a['Extensão'] in ['.xlsx', '.xls']])}")
    
else:
    print("❌ Pasta data/base não encontrada!")

📋 ARQUIVOS DISPONÍVEIS EM data/base
📗 Arquivo Excel
   Nome: plano_de_contas_correto.xlsx
   Tamanho: 0.02 MB
   Modificado: 13/10/2025 14:57

📊 Arquivo CSV
   Nome: yampa_relatorio.csv
   Tamanho: 6.96 MB
   Modificado: 09/09/2025 13:07

📊 Arquivo CSV
   Nome: yampa_relatorio_antigo.csv
   Tamanho: 2.98 MB
   Modificado: 02/11/2022 13:08

📈 RESUMO DOS ARQUIVOS:
                        Nome Extensão  Tamanho (MB)       Modificado                                   Caminho
plano_de_contas_correto.xlsx    .xlsx          0.02 13/10/2025 14:57 ..\data\base\plano_de_contas_correto.xlsx
         yampa_relatorio.csv     .csv          6.96 09/09/2025 13:07          ..\data\base\yampa_relatorio.csv
  yampa_relatorio_antigo.csv     .csv          2.98 02/11/2022 13:08   ..\data\base\yampa_relatorio_antigo.csv

📊 ESTATÍSTICAS:
Total de arquivos: 3
Tamanho total: 9.97 MB
Arquivos CSV: 2
Arquivos Excel: 1


## 4. Carregar e Consolidar Dados

In [7]:
# Carregar dados dos arquivos Yampa
print("📊 CARREGANDO DADOS FINANCEIROS")
print("=" * 50)

# 1. Carregar relatório atual (2022-2025)
try:
    yampa_atual = pd.read_csv('../data/base/yampa_relatorio.csv', sep=';', encoding='utf-8')
    print("✅ Arquivo yampa_relatorio.csv carregado")
    print(f"   📈 Dimensões: {yampa_atual.shape}")
    print(f"   📅 Período estimado: 2022-2025")
except Exception as e:
    print(f"❌ Erro ao carregar yampa_relatorio.csv: {e}")
    yampa_atual = None

# 2. Carregar relatório antigo (2020-2022) - testar diferentes encodings
yampa_antigo = None
encodings = ['latin-1', 'cp1252', 'iso-8859-1', 'utf-8']

for encoding in encodings:
    try:
        yampa_antigo = pd.read_csv('../data/base/yampa_relatorio_antigo.csv', sep=';', encoding=encoding)
        print(f"✅ Arquivo yampa_relatorio_antigo.csv carregado (encoding: {encoding})")
        print(f"   📈 Dimensões: {yampa_antigo.shape}")
        print(f"   📅 Período estimado: 2020-2022")
        break
    except Exception as e:
        continue

if yampa_antigo is None:
    print("❌ Não foi possível carregar yampa_relatorio_antigo.csv")

# 3. Carregar plano de contas
try:
    plano_contas = pd.read_excel('../data/base/plano_de_contas_correto.xlsx')
    print("✅ Arquivo plano_de_contas_correto.xlsx carregado")
    print(f"   📈 Dimensões: {plano_contas.shape}")
    print(f"   📊 Contas cadastradas: {plano_contas.shape[0]}")
except Exception as e:
    print(f"❌ Erro ao carregar plano_de_contas_correto.xlsx: {e}")
    plano_contas = None

# Resumo do carregamento
print(f"\n📋 RESUMO DO CARREGAMENTO:")
print(f"✅ Arquivos carregados com sucesso: {sum([yampa_atual is not None, yampa_antigo is not None, plano_contas is not None])}/3")

if yampa_atual is not None and yampa_antigo is not None:
    total_transacoes = yampa_atual.shape[0] + yampa_antigo.shape[0]
    print(f"💰 Total de transações disponíveis: {total_transacoes:,}")
else:
    print("⚠️ Nem todos os arquivos foram carregados")

📊 CARREGANDO DADOS FINANCEIROS
✅ Arquivo yampa_relatorio.csv carregado
   📈 Dimensões: (27134, 17)
   📅 Período estimado: 2022-2025
✅ Arquivo yampa_relatorio_antigo.csv carregado (encoding: latin-1)
   📈 Dimensões: (19930, 17)
   📅 Período estimado: 2020-2022
✅ Arquivo plano_de_contas_correto.xlsx carregado
   📈 Dimensões: (147, 6)
   📊 Contas cadastradas: 147

📋 RESUMO DO CARREGAMENTO:
✅ Arquivos carregados com sucesso: 3/3
💰 Total de transações disponíveis: 47,064


## 5. Análise Inicial dos Dados

In [5]:
# Análise inicial dos dados carregados
print("🔍 ANÁLISE INICIAL DOS DADOS")
print("=" * 50)

# Analisar arquivo atual
if yampa_atual is not None:
    print("📊 YAMPA RELATÓRIO ATUAL:")
    print(f"   Linhas: {yampa_atual.shape[0]:,}")
    print(f"   Colunas: {yampa_atual.shape[1]}")
    print(f"   Colunas disponíveis: {list(yampa_atual.columns)}")
    
    # Verificar datas
    yampa_atual['Data_realizada'] = pd.to_datetime(yampa_atual['Data realizada'], format='%d/%m/%Y', errors='coerce')
    data_min = yampa_atual['Data_realizada'].min()
    data_max = yampa_atual['Data_realizada'].max()
    print(f"   📅 Período: {data_min.date()} a {data_max.date()}")
    
    # Verificar valores nulos
    nulos_atual = yampa_atual.isnull().sum()
    print(f"   📋 Colunas com valores nulos: {(nulos_atual > 0).sum()}")
    
    print()

# Analisar arquivo antigo
if yampa_antigo is not None:
    print("📊 YAMPA RELATÓRIO ANTIGO:")
    print(f"   Linhas: {yampa_antigo.shape[0]:,}")
    print(f"   Colunas: {yampa_antigo.shape[1]}")
    print(f"   Colunas disponíveis: {list(yampa_antigo.columns)}")
    
    # Verificar datas
    yampa_antigo['Data_realizada'] = pd.to_datetime(yampa_antigo['Data realizada'], format='%d/%m/%Y', errors='coerce')
    data_min_antigo = yampa_antigo['Data_realizada'].min()
    data_max_antigo = yampa_antigo['Data_realizada'].max()
    print(f"   📅 Período: {data_min_antigo.date()} a {data_max_antigo.date()}")
    
    # Verificar valores nulos
    nulos_antigo = yampa_antigo.isnull().sum()
    print(f"   📋 Colunas com valores nulos: {(nulos_antigo > 0).sum()}")
    
    print()

# Analisar plano de contas
if plano_contas is not None:
    print("📊 PLANO DE CONTAS:")
    print(f"   Linhas: {plano_contas.shape[0]:,}")
    print(f"   Colunas: {plano_contas.shape[1]}")
    print(f"   Colunas disponíveis: {list(plano_contas.columns)}")
    
    # Mostrar algumas contas
    print(f"\n   📋 Primeiras 5 contas:")
    display(plano_contas.head())

# Verificar compatibilidade entre arquivos
print("\n🔄 VERIFICAÇÃO DE COMPATIBILIDADE:")

if yampa_atual is not None and yampa_antigo is not None:
    # Comparar colunas
    colunas_atual = set(yampa_atual.columns)
    colunas_antigo = set(yampa_antigo.columns)
    
    colunas_comuns = colunas_atual.intersection(colunas_antigo)
    colunas_diferentes = colunas_atual.symmetric_difference(colunas_antigo)
    
    print(f"   ✅ Colunas em comum: {len(colunas_comuns)}")
    print(f"   ⚠️ Colunas diferentes: {len(colunas_diferentes)}")
    
    if colunas_diferentes:
        print(f"   📋 Diferenças: {list(colunas_diferentes)}")
    
    # Verificar sobreposição de datas
    if data_max_antigo >= data_min:
        print(f"   🔄 Há sobreposição de datas: {data_max_antigo.date()} >= {data_min.date()}")
    else:
        print(f"   ✅ Sem sobreposição de datas")
else:
    print("   ❌ Não é possível verificar compatibilidade")

🔍 ANÁLISE INICIAL DOS DADOS
📊 YAMPA RELATÓRIO ATUAL:
   Linhas: 27,134
   Colunas: 17
   Colunas disponíveis: ['Data esperada', 'Data realizada', 'Plano de contas', 'Histórico', 'Sequência', 'Conta bancária', 'Valor a receber', 'Valor recebido', 'Valor a pagar', 'Valor pago', 'Status', 'Cliente/Fornecedor', 'Meio de pagamento', 'Centro de custo', 'Projeto', 'Data da última alteração', 'Código da conciliação bancária']
   📅 Período: 2022-08-22 a 2025-09-09
   📋 Colunas com valores nulos: 7

📊 YAMPA RELATÓRIO ANTIGO:
   Linhas: 19,930
   Colunas: 17
   Colunas disponíveis: ['Data agendada', 'Data realizada', 'Plano de Conta', 'Historico', 'Sequencia', 'Conta bancaria', 'Valor a receber', 'Valor recebido', 'Valor pago', 'Valor a pagar', 'Status', 'Cliente/Fornecedor', 'Meio de pagamento', 'Centro de custo', 'Projeto', 'Data da ultima alteracao', 'Codigo da conciliacao bancaria']
   📅 Período: 2020-10-01 a 2022-10-31
   📋 Colunas com valores nulos: 6

📊 PLANO DE CONTAS:
   Linhas: 147
   C

,Código,Contas,Categoria,Base do DRE,Despesas Operacionais,COD_DRE
0,3,Receita/Faturamento,RECEITA/FATURAMENTO,Receita Bruta,NaN,1.00
1,31,Receita de vendas,RECEITA/FATURAMENTO,Receita Bruta,NaN,1.00
2,311,Vendas em dinheiro,RECEITA/FATURAMENTO,Receita Bruta,NaN,1.00
3,312,Vendas cartões de débito,RECEITA/FATURAMENTO,Receita Bruta,NaN,1.00
4,313,Vendas cartões de crédito,RECEITA/FATURAMENTO,Receita Bruta,NaN,1.00



🔄 VERIFICAÇÃO DE COMPATIBILIDADE:
   ✅ Colunas em comum: 11
   ⚠️ Colunas diferentes: 14
   📋 Diferenças: ['Plano de contas', 'Código da conciliação bancária', 'Data da ultima alteracao', 'Sequencia', 'Data esperada', 'Histórico', 'Sequência', 'Conta bancária', 'Data da última alteração', 'Conta bancaria', 'Codigo da conciliacao bancaria', 'Data agendada', 'Historico', 'Plano de Conta']
   🔄 Há sobreposição de datas: 2022-10-31 >= 2022-08-22


## 6. Processamento e Consolidação dos Dados

In [ ]:
# Processar e analisar dados financeiros
print("💰 ANÁLISE FINANCEIRA COMPLETA")
print("=" * 60)

# Processar arquivo atual
if yampa_atual is not None:
    # Tratar valores financeiros
    yampa_atual['Valor_pago_num'] = yampa_atual['Valor pago'].apply(tratar_valor_brasileiro)
    yampa_atual['Valor_recebido_num'] = yampa_atual['Valor recebido'].apply(tratar_valor_brasileiro)
    
    # Calcular totais
    total_pago_atual = abs(yampa_atual['Valor_pago_num'].sum())
    total_recebido_atual = yampa_atual['Valor_recebido_num'].sum()
    saldo_atual = total_recebido_atual - total_pago_atual
    
    print(f"📊 PERÍODO ATUAL (2022-2025):")
    print(f"   💸 Total Pago: R$ {total_pago_atual:,.2f}")
    print(f"   💰 Total Recebido: R$ {total_recebido_atual:,.2f}")
    print(f"   📈 Saldo Líquido: R$ {saldo_atual:,.2f}")
    print(f"   📋 Transações: {yampa_atual.shape[0]:,}")

# Processar arquivo antigo
if yampa_antigo is not None:
    # Tratar valores financeiros
    yampa_antigo['Valor_pago_num'] = yampa_antigo['Valor pago'].apply(tratar_valor_brasileiro)
    yampa_antigo['Valor_recebido_num'] = yampa_antigo['Valor recebido'].apply(tratar_valor_brasileiro)
    
    # Calcular totais
    total_pago_antigo = abs(yampa_antigo['Valor_pago_num'].sum())
    total_recebido_antigo = yampa_antigo['Valor_recebido_num'].sum()
    saldo_antigo = total_recebido_antigo - total_pago_antigo
    
    print(f"\n📊 PERÍODO HISTÓRICO (2020-2022):")
    print(f"   💸 Total Pago: R$ {total_pago_antigo:,.2f}")
    print(f"   💰 Total Recebido: R$ {total_recebido_antigo:,.2f}")
    print(f"   📈 Saldo Líquido: R$ {saldo_antigo:,.2f}")
    print(f"   📋 Transações: {yampa_antigo.shape[0]:,}")

# Consolidado
if yampa_atual is not None and yampa_antigo is not None:
    total_pago_consolidado = total_pago_atual + total_pago_antigo
    total_recebido_consolidado = total_recebido_atual + total_recebido_antigo
    saldo_consolidado = total_recebido_consolidado - total_pago_consolidado
    total_transacoes = yampa_atual.shape[0] + yampa_antigo.shape[0]
    
    print(f"\n🎯 CONSOLIDADO (2020-2025):")
    print(f"   💸 Total Pago: R$ {total_pago_consolidado:,.2f}")
    print(f"   💰 Total Recebido: R$ {total_recebido_consolidado:,.2f}")
    print(f"   📈 Saldo Líquido: R$ {saldo_consolidado:,.2f}")
    print(f"   📋 Total Transações: {total_transacoes:,}")
    
    # Métricas adicionais
    print(f"\n📊 MÉTRICAS FINANCEIRAS:")
    if total_recebido_consolidado > 0:
        margem = (saldo_consolidado / total_recebido_consolidado) * 100
        print(f"   📈 Margem Líquida: {margem:.2f}%")
    
    ticket_medio = (total_recebido_consolidado + total_pago_consolidado) / total_transacoes
    print(f"   🎫 Ticket Médio: R$ {ticket_medio:,.2f}")
    
    # Média mensal (5 anos = 60 meses)
    receita_mensal = total_recebido_consolidado / 60
    despesa_mensal = total_pago_consolidado / 60
    print(f"   📅 Receita Média Mensal: R$ {receita_mensal:,.2f}")
    print(f"   📅 Despesa Média Mensal: R$ {despesa_mensal:,.2f}")

In [ ]:
# Processar e consolidar dados financeiros
print("🔄 PROCESSAMENTO E CONSOLIDAÇÃO DOS DADOS")
print("=" * 60)

# Processar arquivo atual
if yampa_atual is not None:
    print("📊 Processando arquivo atual...")
    
    # Tratar valores financeiros
    yampa_atual['Valor_pago'] = yampa_atual['Valor pago'].apply(tratar_valor_brasileiro)
    yampa_atual['Valor_recebido'] = yampa_atual['Valor recebido'].apply(tratar_valor_brasileiro)
    yampa_atual['Valor_a_pagar'] = yampa_atual['Valor a pagar'].apply(tratar_valor_brasileiro)
    yampa_atual['Valor_a_receber'] = yampa_atual['Valor a receber'].apply(tratar_valor_brasileiro)
    
    # Criar coluna de valor líquido (receitas - despesas)
    yampa_atual['Valor_liquido'] = yampa_atual['Valor_recebido'] - yampa_atual['Valor_pago']
    
    # Adicionar flag de origem
    yampa_atual['Fonte'] = 'Atual'
    
    print(f"   ✅ {yampa_atual.shape[0]:,} transações processadas")

# Processar arquivo antigo
if yampa_antigo is not None:
    print("📊 Processando arquivo antigo...")
    
    # Tratar valores financeiros
    yampa_antigo['Valor_pago'] = yampa_antigo['Valor pago'].apply(tratar_valor_brasileiro)
    yampa_antigo['Valor_recebido'] = yampa_antigo['Valor recebido'].apply(tratar_valor_brasileiro)
    yampa_antigo['Valor_a_pagar'] = yampa_antigo['Valor a pagar'].apply(tratar_valor_brasileiro)
    yampa_antigo['Valor_a_receber'] = yampa_antigo['Valor a receber'].apply(tratar_valor_brasileiro)
    
    # Criar coluna de valor líquido
    yampa_antigo['Valor_liquido'] = yampa_antigo['Valor_recebido'] - yampa_antigo['Valor_pago']
    
    # Adicionar flag de origem
    yampa_antigo['Fonte'] = 'Antigo'
    
    print(f"   ✅ {yampa_antigo.shape[0]:,} transações processadas")

# Consolidar dados se ambos estiverem disponíveis
dados_consolidados = None

if yampa_atual is not None and yampa_antigo is not None:
    print("\n🔄 Consolidando dados...")
    
    # Padronizar nomes de colunas
    colunas_padrao = {
        'Data realizada': 'Data_realizada',
        'Plano de contas': 'Plano_contas',
        'Plano de Conta': 'Plano_contas',
        'Histórico': 'Historico',
        'Historico': 'Historico',
        'Cliente/Fornecedor': 'Cliente_Fornecedor',
        'Centro de custo': 'Centro_custo',
        'Meio de pagamento': 'Meio_pagamento'
    }
    
    # Renomear colunas
    yampa_atual_renamed = yampa_atual.rename(columns=colunas_padrao)
    yampa_antigo_renamed = yampa_antigo.rename(columns=colunas_padrao)
    
    # Selecionar colunas comuns
    colunas_importantes = [
        'Data_realizada', 'Plano_contas', 'Historico', 'Valor_pago', 
        'Valor_recebido', 'Valor_liquido', 'Status', 'Cliente_Fornecedor',
        'Meio_pagamento', 'Centro_custo', 'Fonte'
    ]
    
    # Manter apenas colunas que existem em ambos
    colunas_disponiveis = []
    for col in colunas_importantes:
        if col in yampa_atual_renamed.columns and col in yampa_antigo_renamed.columns:
            colunas_disponiveis.append(col)
    
    # Consolidar
    try:
        dados_consolidados = pd.concat([
            yampa_atual_renamed[colunas_disponiveis],
            yampa_antigo_renamed[colunas_disponiveis]
        ], ignore_index=True)
        
        print(f"   ✅ Dados consolidados: {dados_consolidados.shape[0]:,} transações")
        print(f"   📊 Colunas consolidadas: {len(colunas_disponiveis)}")
        
        # Ordenar por data
        dados_consolidados = dados_consolidados.sort_values('Data_realizada').reset_index(drop=True)
        
        # Calcular resumo financeiro
        total_receitas = dados_consolidados['Valor_recebido'].sum()
        total_despesas = dados_consolidados['Valor_pago'].sum()
        saldo_liquido = total_receitas - total_despesas
        
        print(f"\n💰 RESUMO FINANCEIRO CONSOLIDADO:")
        print(f"   💚 Total de receitas: R$ {total_receitas:,.2f}")
        print(f"   🔴 Total de despesas: R$ {abs(total_despesas):,.2f}")
        print(f"   💰 Saldo líquido: R$ {saldo_liquido:,.2f}")
        
        # Período completo
        data_inicio = dados_consolidados['Data_realizada'].min()
        data_fim = dados_consolidados['Data_realizada'].max()
        print(f"   📅 Período total: {data_inicio.date()} a {data_fim.date()}")
        
    except Exception as e:
        print(f"   ❌ Erro na consolidação: {e}")

elif yampa_atual is not None:
    dados_consolidados = yampa_atual
    print("   ⚠️ Usando apenas dados atuais")
elif yampa_antigo is not None:
    dados_consolidados = yampa_antigo
    print("   ⚠️ Usando apenas dados antigos")
else:
    print("   ❌ Nenhum dado disponível para consolidação")